<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [25]:
import praw
import pandas as pd
import re

import requests
import time
import sys
import pdb

In [24]:
rdb = pd.read_parquet('redditwhiskeydatabase.parquet')

/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


Client secrets for reddit are loaded here. You'll need to provide your own secrets.py which provides the following fields. You'll need to sign up for a reddit api to get this.:

- clientid
- clientsecret
- clientusername
- clientpassword

In [28]:
import secrets

In [44]:
def extract_reviews(dataframe,re_client_id,re_client_secret,\
                    re_username,re_pw):
        """Takes Reddit review submission ids, request \
        submissions from PRAW, search for the right body text,\
        and saves it to the dataframe""" 
        #timer/counter
        start_time = int(time.time())
        count = 0
        total = dataframe.shape[0]
        
        #get the Review URL addresses as a series
        rev_addr = dataframe['sub_id']

        #empty list of reviews
        reviews = []
        
        #initiate PRAW
        reddit = praw.Reddit(client_id= re_client_id,\
                     client_secret = re_client_secret,\
                     username = re_username, \
                             password = re_pw,user_agent= 'v1' )
        
        
        
        #iterate through the submission ids to extract review texts
        for s_id in rev_addr:
            review = ''
            try:
                sub = reddit.submission(id = s_id)
                s_author = sub.author.name
            except:
                reviews.append('error')
                continue
            #sometimes the reviews are in the 
            #submission text instead of comments
            #add any text in this field if not empty
            sub_text = sub.selftext
            if len(sub_text) > 0:
                review = review + sub_text + ' \n'

            # parse through the submission's comments 
            # and extract all of authors' comments
            for comment in sub.comments:
                try:
                    if comment.author.name == s_author:
                        review = review + comment.body + '\n'
                except:
                    continue
            
            reviews.append(review)
            
            #counter/timer
            end_time = int(time.time())
            min_spent = round(((end_time-start_time)/60),2)
            remaining_time = round((min_spent/max(count,1))*max((total-count),1),2)
            
            
            count_str = 'extracted ' + str(count) + '/' + str(total) + \
                    ' reviews in total ' + str(min_spent) + ' minutes' + \
                    ' estimated time remaining is ' + str(remaining_time) + ' minutes'
            sys.stdout.write("\r{}".format(count_str))
            count += 1
            
        dataframe['review'] = reviews
        #save to csv, also return the pandas dataframe
        dataframe.to_csv("db_reviews.csv")
        return dataframe
        

In [46]:
reviews = extract_reviews(rdb,clientid,clientsecret,\
                    clientusername,clientpassword)

extracted 14049/31282 reviews in total 252.53 minutes estimated time remaining is 309.76 minutes

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



extracted 18304/31282 reviews in total 330.1 minutes estimated time remaining is 234.05 minutess

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



extracted 22988/31282 reviews in total 414.83 minutes estimated time remaining is 149.67 minutes

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



extracted 27449/31282 reviews in total 509.53 minutes estimated time remaining is 71.15 minutess

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



extracted 29572/31282 reviews in total 546.32 minutes estimated time remaining is 31.59 minutes

In [47]:
reviews.shape

(31282, 10)

In [50]:
reviews.to_parquet('db_reviews.parquet')

/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [51]:
reviews.tail()

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date,sub_id,review
reviewID,,,,,,,,,,
31887,2019-07-09 01:02:43.534,Compass Box Great King St. Experimental Batch ...,implar,https://www.reddit.com/r/Scotch/comments/cawtf...,86,Blend,nan,2019-07-09,cawtft,"I'm quite late with this review, as it's not t..."
31888,2019-07-09 05:28:10.934,Heaven Hill 9 North Star,TOModera,https://old.reddit.com/r/bourbon/comments/caz2...,84,Kentucky,nan,2019-07-09,caz2cx,"Every so often, I’m reminded of my most popula..."
31889,2019-07-09 08:51:55.307,Ardbeg 17 Elements of Islay - Ar10,devoz,https://www.reddit.com/r/Scotch/comments/cb151...,89,Islay,nan,2019-07-09,cb1518,Review 547: Ardbeg 17 (Elements of Islay - AR1...
31890,2019-07-09 17:05:19.335,Highland Park 16 SMWS 4.250,HawkI84,https://www.reddit.com/r/Scotch/comments/cb7mq...,85,Scotch,nan,2019-07-09,cb7mql,u/xreekinghavocx\nThis is a 16 year old Highla...
31891,2019-07-09 17:12:56.609,Tomintoul 14,razzafrachen,https://www.reddit.com/r/Scotch/comments/cb7oa...,86,Speyside,40,2019-07-09,cb7oab,"*Ah yes, Tomintoul 14. A ubiquitous bottle in ..."
